## [Issue #7](https://github.com/yeemey/haackwell/issues/7)

### Map pattern of polluted, damaged, stolen, etc.

1. Split out free text data from 'STATUS' column into more specific columns.
2. Apply polluted etc. trends to map.

#### Questions

1. Are mechanical pump failures/pollution related to pump type/make?

In [2]:
import pandas as pd
import re
import matplotlib
import matplotlib.pyplot as plt

In [3]:
# Water Point Data Exchange
wpdx_data = 'https://www.hydroshare.org/django_irods/download/d3659dcf575d4090801a74d1ce096d7c/data/contents/WPDx_Well_Function_Upd_151224_xy161117.csv'
wpdx_df = pd.read_csv(wpdx_data)
well_bkdown_df = pd.read_csv('/Users/ymseah/Google Drive/GeoHackWeek/haackwell/dat/well-data-2001-2015-no-rainwater.csv')

In [4]:
wpdx_df

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,COD_FCN,COD_QTY,COD_RESRCE,ADM1,ADM2,ACTIVITY,COUNTRY,WATERSRC,WATERTECH,INSTALLED,MGMT,PAY,SOURCE,RPT_DATE
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,zmbpw,LR,Manual pump on hand-dug well,Vergnet,0,NaN,No water committee,WASH Liberia,21/01/2011
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,z4ja5,LR,Manual pump on hand-dug well,Vergnet,1986,NaN,No water committee,WASH Liberia,21/01/2011
2,357349,5.802157,-9.645714,Yes,Working but with problems. Not priming,2,1,0,Rivercess,Norwein,yjryl,LR,Manual pump on hand-dug well,Afridev,2008,NaN,No water committee,WASH Liberia,02/02/2011
3,489514,-0.541100,34.375820,No,Drought|No operation in the dry season,999,0,1,Homa Bay,Mbita,Yao Oinga,KE,NaN,Surface water,0,0,No payment system,Engineering Sciences & Global Development,24/01/2011
4,357595,5.716055,-9.618187,No,Broken Down System. low water table,0,1,1,Rivercess,Norwein,y1ccy,LR,Manual pump on hand-dug well,Afridev,2005,NaN,No water committee,WASH Liberia,03/02/2011
5,489266,-0.730000,34.366000,No,No fuel|No operation at least once a week,2,1,0,Homa Bay,Ndhiwa,Water Kiosk,KE,NaN,Gravity-fed communal standpipe,1995,Private Operator/Delegated Management,Per Bucket,Engineering Sciences & Global Development,18/02/2011
6,489625,-0.450333,34.009880,No,Low yield|No operation in the dry season,2,1,1,Homa Bay,Mbita,Wakula Dispensary BH,KE,NaN,Borehole with hand pump,0,Institutional Management,No payment system,Engineering Sciences & Global Development,02/10/2011
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,2,1,0,Sinoe,Sanquin Dist#2,v5dph,LR,Manual pump on hand-dug well,Afridev,2010,Community Management,Yes but only in case of breakdown,WASH Liberia,27/01/2011
8,361779,5.225134,-8.121493,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,v42mh,LR,Manual pump on hand-dug well,Afridev,2009,NaN,No water committee,WASH Liberia,18/02/2011
9,361780,5.225422,-8.119787,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,v42mf,LR,Manual pump on hand-dug well,Afridev,2009,NaN,No water committee,WASH Liberia,18/02/2011


In [5]:
well_status_df = wpdx_df[['WELL_ID', 'LAT_DD', 'LONG_DD', 'FUNC', 'STATUS', 'WATERSRC', 'WATERTECH', 'INSTALLED']].copy()
well_status_df

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,WATERSRC,WATERTECH,INSTALLED
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,Manual pump on hand-dug well,Vergnet,0
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,Manual pump on hand-dug well,Vergnet,1986
2,357349,5.802157,-9.645714,Yes,Working but with problems. Not priming,Manual pump on hand-dug well,Afridev,2008
3,489514,-0.541100,34.375820,No,Drought|No operation in the dry season,NaN,Surface water,0
4,357595,5.716055,-9.618187,No,Broken Down System. low water table,Manual pump on hand-dug well,Afridev,2005
5,489266,-0.730000,34.366000,No,No fuel|No operation at least once a week,NaN,Gravity-fed communal standpipe,1995
6,489625,-0.450333,34.009880,No,Low yield|No operation in the dry season,NaN,Borehole with hand pump,0
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,Manual pump on hand-dug well,Afridev,2010
8,361779,5.225134,-8.121493,Yes,Working but with problems. insufficient water,Manual pump on hand-dug well,Afridev,2009
9,361780,5.225422,-8.119787,Yes,Working but with problems. insufficient water,Manual pump on hand-dug well,Afridev,2009


In [6]:
well_status_df['STATUS'].value_counts()

DRY                                                                                                                           3708
Status:Functional|Quantity:Insufficient|Quality:Soft                                                                          3323
Functional ( in use)                                                                                                          2971
Status:Not functional|Quantity:Insufficient|Quality:Soft                                                                      1572
Status:Functional|Quantity:Seasonal|Quality:Soft                                                                               394
Status:Not functional|Quantity:Dry|Quality:Soft                                                                                305
Status:Functional|Quantity:Insufficient|Quality:Salty                                                                          262
Status:Not functional|Breakdown Year:2003|Quantity:Insufficient|Quality:Soft       

In [7]:
well_status = zip(well_status_df['WELL_ID'].tolist(), well_status_df['STATUS'].tolist())
well_status_dict = {}
for well_id, status in well_status:
    if re.search('Status:Not functional', status) or re.search('Status:Functional', status):
        status_dict = {x.split(':')[0]: x.split(':')[1] for x in status.split('|')}
        well_status_dict[well_id] = status_dict
print(well_status_dict)

{267515: {'Status': 'Functional', 'Breakdown Year': '1978', 'Reason Not Functioning': 'Abandoned pumping scheme', 'Quantity': 'Dry', 'Quality': 'Soft'}, 276237: {'Status': 'Functional', 'Breakdown Year': '1978', 'Reason Not Functioning': 'Replacement of pump machine', 'Quantity': 'Insufficient', 'Quality': 'Soft'}, 276235: {'Status': 'Functional', 'Breakdown Year': '1978', 'Reason Not Functioning': 'Replacement of pump machine', 'Quantity': 'Insufficient', 'Quality': 'Soft'}, 276241: {'Status': 'Functional', 'Breakdown Year': '1978', 'Reason Not Functioning': 'Replacement of pump machine', 'Quantity': 'Insufficient', 'Quality': 'Soft'}, 276286: {'Status': 'Functional', 'Breakdown Year': '1978', 'Reason Not Functioning': 'Replacement of pump machine', 'Quantity': 'Insufficient', 'Quality': 'Soft'}, 276283: {'Status': 'Functional', 'Breakdown Year': '1978', 'Reason Not Functioning': 'Replacement of pump machine', 'Quantity': 'Insufficient', 'Quality': 'Soft'}, 276285: {'Status': 'Functio

In [8]:
col_names = []
for value in well_status_dict.values():
    for key in value.keys():
        if key not in col_names:
            col_names.append(key)
for column in col_names:
    well_status_df[column] = ''

In [9]:
for key, value in well_status_dict.items():
    for col in value.keys():
        row_index = well_status_df.loc[well_status_df['WELL_ID'] == key].index[0]
        well_status_df.set_value(row_index, col, value[col])

In [10]:
well_status_df[well_status_df['Status'] == 'Not functional']

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,WATERSRC,WATERTECH,INSTALLED,Status,Breakdown Year,Reason Not Functioning,Quantity,Quality
13912,283493,-10.780543,39.550040,No,Status:Not functional|Breakdown Year:1961|Quan...,Machine Drilled Borehole,Submersible Communal standpipe multiple,0,Not functional,1961,,Insufficient,Soft
13913,280685,-1.980408,33.853328,No,Status:Not functional|Breakdown Year:1967|Quan...,Lake,KSB Communal standpipe multiple,1967,Not functional,1967,,Insufficient,Soft
13914,279288,-3.771674,37.714323,No,Status:Not functional|Breakdown Year:1970|Reas...,River/Lake,Gravity Communal standpipe,1972,Not functional,1970,Abandoned,Dry,Soft
13915,284534,-2.619040,33.389823,No,Status:Not functional|Breakdown Year:1975|Quan...,Lake,Submersible Communal standpipe multiple,1974,Not functional,1975,,Insufficient,Soft
13916,281463,-9.024967,33.031670,No,Status:Not functional|Breakdown Year:1975|Quan...,River/Lake,None Communal standpipe,1975,Not functional,1975,,Seasonal,Soft
13917,275143,-1.276619,31.758540,No,Status:Not functional|Breakdown Year:1976|Quan...,Spring,Gravity Communal standpipe multiple,1973,Not functional,1976,,Enough,Soft
13918,275277,-1.254512,31.808250,No,Status:Not functional|Breakdown Year:1976|Quan...,River,None Communal standpipe multiple,1973,Not functional,1976,,Enough,Soft
13919,275241,-1.410368,31.786083,No,Status:Not functional|Breakdown Year:1976|Quan...,River,None Communal standpipe multiple,1973,Not functional,1976,,Enough,Soft
13920,275030,-1.426130,31.766791,No,Status:Not functional|Breakdown Year:1976|Quan...,River,None Communal standpipe multiple,1973,Not functional,1976,,Enough,Soft
13921,275276,-1.254429,31.811540,No,Status:Not functional|Breakdown Year:1976|Quan...,River,None Communal standpipe multiple,1973,Not functional,1976,,Enough,Soft


In [11]:
well_status_df[well_status_df['Status'] == 'Functional']

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,WATERSRC,WATERTECH,INSTALLED,Status,Breakdown Year,Reason Not Functioning,Quantity,Quality
9001,267515,-3.411438,35.588334,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Rainwater harvesting,Gravity Communal standpipe,2006,Functional,1978,Abandoned pumping scheme,Dry,Soft
9002,276237,-1.790448,31.537909,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Spring,Gravity Improved spring,1989,Functional,1978,Replacement of pump machine,Insufficient,Soft
9003,276235,-1.786858,31.539250,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Spring,Gravity Improved spring,2002,Functional,1978,Replacement of pump machine,Insufficient,Soft
9004,276241,-1.799906,31.530007,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Shallow well,Nira/Tanira Hand pump,1997,Functional,1978,Replacement of pump machine,Insufficient,Soft
9005,276286,-1.790337,31.584378,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Rainwater harvesting,None Communal standpipe,2001,Functional,1978,Replacement of pump machine,Insufficient,Soft
9006,276283,-1.783856,31.576263,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Rainwater harvesting,None Communal standpipe,2002,Functional,1978,Replacement of pump machine,Insufficient,Soft
9007,276285,-1.787038,31.561982,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Machine-drilled borehole,India mark II Hand pump,2001,Functional,1978,Replacement of pump machine,Insufficient,Soft
9008,276336,-1.789202,31.556789,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Shallow well,Nira/Tanira Hand pump,1999,Functional,1978,Replacement of pump machine,Insufficient,Soft
9009,276282,-1.783413,31.569716,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Rainwater harvesting,None Communal standpipe multiple,2001,Functional,1978,Replacement of pump machine,Insufficient,Soft
9010,276284,-1.786728,31.542469,Yes,Status:Functional|Breakdown Year:1978|Reason N...,Spring,Gravity Improved spring,2002,Functional,1978,Replacement of pump machine,Insufficient,Soft


In [12]:
well_status_subset = well_status_df[['WELL_ID', 'Status', 'Breakdown Year', 'Reason Not Functioning', 'Quantity', 'Quality']].copy()
new_wpdx_df = wpdx_df.merge(well_status_subset, how='left', on='WELL_ID')
new_wpdx_df

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,COD_FCN,COD_QTY,COD_RESRCE,ADM1,ADM2,...,INSTALLED,MGMT,PAY,SOURCE,RPT_DATE,Status,Breakdown Year,Reason Not Functioning,Quantity,Quality
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,0,NaN,No water committee,WASH Liberia,21/01/2011,,,,,
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,1986,NaN,No water committee,WASH Liberia,21/01/2011,,,,,
2,357349,5.802157,-9.645714,Yes,Working but with problems. Not priming,2,1,0,Rivercess,Norwein,...,2008,NaN,No water committee,WASH Liberia,02/02/2011,,,,,
3,489514,-0.541100,34.375820,No,Drought|No operation in the dry season,999,0,1,Homa Bay,Mbita,...,0,0,No payment system,Engineering Sciences & Global Development,24/01/2011,,,,,
4,357595,5.716055,-9.618187,No,Broken Down System. low water table,0,1,1,Rivercess,Norwein,...,2005,NaN,No water committee,WASH Liberia,03/02/2011,,,,,
5,489266,-0.730000,34.366000,No,No fuel|No operation at least once a week,2,1,0,Homa Bay,Ndhiwa,...,1995,Private Operator/Delegated Management,Per Bucket,Engineering Sciences & Global Development,18/02/2011,,,,,
6,489625,-0.450333,34.009880,No,Low yield|No operation in the dry season,2,1,1,Homa Bay,Mbita,...,0,Institutional Management,No payment system,Engineering Sciences & Global Development,02/10/2011,,,,,
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,2,1,0,Sinoe,Sanquin Dist#2,...,2010,Community Management,Yes but only in case of breakdown,WASH Liberia,27/01/2011,,,,,
8,361779,5.225134,-8.121493,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,...,2009,NaN,No water committee,WASH Liberia,18/02/2011,,,,,
9,361780,5.225422,-8.119787,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,...,2009,NaN,No water committee,WASH Liberia,18/02/2011,,,,,


In [13]:
new_wpdx_df[new_wpdx_df['Status'] == 'Functional']

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,COD_FCN,COD_QTY,COD_RESRCE,ADM1,ADM2,...,INSTALLED,MGMT,PAY,SOURCE,RPT_DATE,Status,Breakdown Year,Reason Not Functioning,Quantity,Quality
9001,267515,-3.411438,35.588334,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Arusha,Karatu,...,2006,NaN,Pay per bucket,SNV,10/06/2008,Functional,1978,Abandoned pumping scheme,Dry,Soft
9002,276237,-1.790448,31.537909,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,1989,NaN,Pay when scheme fails,SNV,21/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9003,276235,-1.786858,31.539250,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,2002,NaN,Pay when scheme fails,SNV,21/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9004,276241,-1.799906,31.530007,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,1997,NaN,Pay when scheme fails,SNV,21/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9005,276286,-1.790337,31.584378,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,2001,NaN,Pay when scheme fails,SNV,20/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9006,276283,-1.783856,31.576263,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,2002,NaN,Pay when scheme fails,SNV,20/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9007,276285,-1.787038,31.561982,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,2001,NaN,Pay when scheme fails,SNV,18/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9008,276336,-1.789202,31.556789,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,1999,NaN,Pay when scheme fails,SNV,29/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9009,276282,-1.783413,31.569716,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,2001,NaN,Pay when scheme fails,SNV,20/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft
9010,276284,-1.786728,31.542469,Yes,Status:Functional|Breakdown Year:1978|Reason N...,1,1,0,Kagera,Muleba,...,2002,NaN,Pay when scheme fails,SNV,20/11/2007,Functional,1978,Replacement of pump machine,Insufficient,Soft


In [15]:
# temp.csv generated in haackwell/notebooks/extract_status_todummy.ipynb
# list of words used to identify mechanical failure from 'STATUS' entries: 
# mech = ['pump', 'handle', 'pipes', 'tank', 'construction', 'cylinder', 'apron', 'repair', 'parts', 'installation', 'broken', 'blocked', 'technical']

nltk_mechfail_df = pd.read_csv('/Users/ymseah/Google Drive/GeoHackWeek/haackwell/dat/temp.csv')
nltk_mechfail_df

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,COD_FCN,COD_QTY,COD_RESRCE,ADM1,ADM2,...,WATERSRC,WATERTECH,INSTALLED,MGMT,PAY,SOURCE,RPT_DATE,RPT_YEAR,BKDWN_YEAR,mechfail
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,Manual pump on hand-dug well,Vergnet,0,NaN,No water committee,WASH Liberia,2011-01-21,2011.0,2011.0,1.0
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,Manual pump on hand-dug well,Vergnet,1986,NaN,No water committee,WASH Liberia,2011-01-21,2011.0,2011.0,1.0
2,357349,5.802157,-9.645714,Yes,Working but with problems. Not priming,2,1,0,Rivercess,Norwein,...,Manual pump on hand-dug well,Afridev,2008,NaN,No water committee,WASH Liberia,2011-02-02,2011.0,2011.0,0.0
3,489514,-0.541100,34.375820,No,Drought|No operation in the dry season,999,0,1,Homa Bay,Mbita,...,Not recorded,Surface water,0,0,No payment system,Engineering Sciences & Global Development,2011-01-24,2011.0,2011.0,0.0
4,357595,5.716055,-9.618187,No,Broken Down System. low water table,0,1,1,Rivercess,Norwein,...,Manual pump on hand-dug well,Afridev,2005,NaN,No water committee,WASH Liberia,2011-03-02,2011.0,2011.0,1.0
5,489266,-0.730000,34.366000,No,No fuel|No operation at least once a week,2,1,0,Homa Bay,Ndhiwa,...,Not recorded,Gravity-fed communal standpipe,1995,Private Operator/Delegated Management,Per Bucket,Engineering Sciences & Global Development,2011-02-18,2011.0,2011.0,0.0
6,489625,-0.450333,34.009880,No,Low yield|No operation in the dry season,2,1,1,Homa Bay,Mbita,...,Not recorded,Borehole with hand pump,0,Institutional Management,No payment system,Engineering Sciences & Global Development,2011-02-10,2011.0,2011.0,0.0
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,2,1,0,Sinoe,Sanquin Dist#2,...,Manual pump on hand-dug well,Afridev,2010,Community Management,Yes but only in case of breakdown,WASH Liberia,2011-01-27,2011.0,2011.0,1.0
8,361779,5.225134,-8.121493,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,...,Manual pump on hand-dug well,Afridev,2009,NaN,No water committee,WASH Liberia,2011-02-18,2011.0,2011.0,0.0
9,361780,5.225422,-8.119787,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,...,Manual pump on hand-dug well,Afridev,2009,NaN,No water committee,WASH Liberia,2011-02-18,2011.0,2011.0,0.0


In [16]:
wpdx_mechfail_df = new_wpdx_df.merge(nltk_mechfail_df[['WELL_ID', 'mechfail']].copy(), how='left', on='WELL_ID')

In [17]:
wpdx_mechfail_df

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,COD_FCN,COD_QTY,COD_RESRCE,ADM1,ADM2,...,MGMT,PAY,SOURCE,RPT_DATE,Status,Breakdown Year,Reason Not Functioning,Quantity,Quality,mechfail
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,NaN,No water committee,WASH Liberia,21/01/2011,,,,,,1.0
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,NaN,No water committee,WASH Liberia,21/01/2011,,,,,,1.0
2,357349,5.802157,-9.645714,Yes,Working but with problems. Not priming,2,1,0,Rivercess,Norwein,...,NaN,No water committee,WASH Liberia,02/02/2011,,,,,,0.0
3,489514,-0.541100,34.375820,No,Drought|No operation in the dry season,999,0,1,Homa Bay,Mbita,...,0,No payment system,Engineering Sciences & Global Development,24/01/2011,,,,,,0.0
4,357595,5.716055,-9.618187,No,Broken Down System. low water table,0,1,1,Rivercess,Norwein,...,NaN,No water committee,WASH Liberia,03/02/2011,,,,,,1.0
5,489266,-0.730000,34.366000,No,No fuel|No operation at least once a week,2,1,0,Homa Bay,Ndhiwa,...,Private Operator/Delegated Management,Per Bucket,Engineering Sciences & Global Development,18/02/2011,,,,,,0.0
6,489625,-0.450333,34.009880,No,Low yield|No operation in the dry season,2,1,1,Homa Bay,Mbita,...,Institutional Management,No payment system,Engineering Sciences & Global Development,02/10/2011,,,,,,0.0
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,2,1,0,Sinoe,Sanquin Dist#2,...,Community Management,Yes but only in case of breakdown,WASH Liberia,27/01/2011,,,,,,1.0
8,361779,5.225134,-8.121493,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,...,NaN,No water committee,WASH Liberia,18/02/2011,,,,,,0.0
9,361780,5.225422,-8.119787,Yes,Working but with problems. insufficient water,2,1,0,River Gee,Karforh,...,NaN,No water committee,WASH Liberia,18/02/2011,,,,,,0.0


In [29]:
wpdx_mechfail_df[(wpdx_mechfail_df['FUNC'] == 'Yes') & (wpdx_mechfail_df['mechfail'] == 1.0)]

,WELL_ID,LAT_DD,LONG_DD,FUNC,STATUS,COD_FCN,COD_QTY,COD_RESRCE,ADM1,ADM2,...,MGMT,PAY,SOURCE,RPT_DATE,Status,Breakdown Year,Reason Not Functioning,Quantity,Quality,mechfail
0,362092,5.982436,-8.180609,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,NaN,No water committee,WASH Liberia,21/01/2011,,,,,,1.0
1,362100,5.899207,-8.173315,Yes,Working but with problems. Well polluted|Under...,2,1,0,Grand Gedeh,Tchien,...,NaN,No water committee,WASH Liberia,21/01/2011,,,,,,1.0
7,364570,5.231378,-9.141873,Yes,Working but with problems. Well polluted|Under...,2,1,0,Sinoe,Sanquin Dist#2,...,Community Management,Yes but only in case of breakdown,WASH Liberia,27/01/2011,,,,,,1.0
19,366548,6.328716,-10.755411,Yes,Working but with problems. Apron damaged|sticking,2,1,0,Montserrado,Greater Monrovia,...,Other,No water committee,WASH Liberia,31/05/2011,,,,,,1.0
33,362882,4.386985,-7.718236,Yes,Working but with problems. Well polluted|Under...,2,1,0,Maryland,Harper,...,Community Management,Yes but only in case of breakdown,WASH Liberia,24/01/2011,,,,,,1.0
426,486508,-0.133523,34.978078,Yes,Functional ( in use)|New Under construction,2,1,0,NaN,NaN,...,Community Management,None,Virtual Kenya,21/08/2012,,,,,,1.0
1082,484693,-2.517898,36.861451,Yes,Functional ( in use)|New Under construction,2,1,0,NaN,NaN,...,Community Management,After System Breakdown,Virtual Kenya,16/10/2012,,,,,,1.0
1320,488930,-1.011242,36.903590,Yes,Functional ( in use)|New Under construction,2,1,0,NaN,NaN,...,Institutional Management,None,Virtual Kenya,22/10/2012,,,,,,1.0
1682,304312,9.861339,-11.428113,Yes,Yes- but partly damaged. No water in network o...,2,1,0,Northern,Koinadugu,...,NaN,Only if there is a breakdown,Sierra Leone STATWASH portal,28/02/2012,,,,,,1.0
1688,305547,9.595845,-11.554954,Yes,Yes- but partly damaged. No water in tank,2,1,0,Northern,Koinadugu,...,NaN,No- water is free,Sierra Leone STATWASH portal,01/03/2012,,,,,,1.0
